<a href="https://colab.research.google.com/github/emd-aquila/Xc51-MOFs/blob/main/run_unimof_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is a test file to send to github

test = "true"